This is the main notebook I use for testing code and creating graphs

In [61]:
# check pytorch version
import torch
print(torch.__version__)

2.5.1+cpu


In [179]:
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid, TUDataset
from torch_geometric.nn import GCN
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.metrics import accuracy_score
from torch.nn import ReLU, Linear
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_networkx
from torch_geometric.nn import GINConv, global_add_pool, Sequential
from torch_geometric.nn import GATConv, GPSConv, GINEConv
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import OneHotDegree
from torch.optim import Adam

In [63]:
#Import the datasets
cora = Planetoid(root='/tmp/Cora', name='Cora')
imdb = TUDataset(root=f'/tmp/IMDB-BINARY', name='IMDB-BINARY')
enzymes = TUDataset(root=f'/tmp/ENZYMES', name='ENZYMES')

In [64]:
#Define a dict that we will use to store performance for each of our models on each dataset
scores = {"Cora": {}, "Imdb": {}, "Enzymes": {}}

# GCN Performance

#### Node classification

In [65]:
#Define a basic GCN for node classification
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(cora.num_node_features, 16)
        self.conv2 = GCNConv(16, cora.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [66]:
# Get the model, data, and optimizer setup in torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = cora[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [67]:
# train model for 150 epochs

model.train()
for epoch in range(151):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')

Epoch 000, Loss: 1.9454
Epoch 020, Loss: 0.1078
Epoch 040, Loss: 0.0161
Epoch 060, Loss: 0.0151
Epoch 080, Loss: 0.0174
Epoch 100, Loss: 0.0161
Epoch 120, Loss: 0.0143
Epoch 140, Loss: 0.0129


In [68]:
#evaluate the model
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
scores["Cora"]["GCN"] = acc
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8010


#### Graph Classification

In [69]:
#Define a basic GCN for graph classification
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, 16)
        self.fc = torch.nn.Linear(16, num_classes)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

In [70]:
# Helper fuction to make node features for the IMDB dataset
# We just represent each node with its degree
def create_degree_features(data):
    num_nodes = data.num_nodes
    degrees = torch.bincount(data.edge_index[0], minlength=num_nodes)
    
    feature_vector = degrees.view(-1, 1).float()
    
    return feature_vector

In [71]:
# Helper function to load data, train the model, and evaluate it for graph classification
def train_and_evaluate(dataset, epochs = 150):

    # Loading Data
    train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GCN(dataset.num_node_features or 1, dataset.num_classes).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Training
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()

            # if no node features we add in the degrees
            if data.x is None:
                data.x = create_degree_features(data).to(device)

            out = model(data.x, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        if epoch % 20 == 0:
            print(f'Epoch {epoch}, Loss: {loss:.4f}')

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    for data in test_loader:
        data = data.to(device)

        # if no node features we add in the degrees
        if data.x is None:
            data.x = create_degree_features(data).to(device)

        out = model(data.x, data.edge_index, data.batch)
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())
        total += data.num_graphs

    accuracy = correct / total
    print(f'Accuracy: {accuracy:.4f}')
    
    return accuracy

In [72]:
# Test model on Imdb dataset

In [73]:
scores["Imdb"]["GCN"] = train_and_evaluate(imdb)

Epoch 0, Loss: 0.6883
Epoch 20, Loss: 0.5198
Epoch 40, Loss: 0.6484
Epoch 60, Loss: 0.4552
Epoch 80, Loss: 0.4928
Epoch 100, Loss: 0.5091
Epoch 120, Loss: 0.4440
Epoch 140, Loss: 0.5197
Accuracy: 0.7100


In [74]:
# Test model on Enzymes dataset

In [75]:
scores["Enzymes"]["GCN"] = train_and_evaluate(enzymes)

Epoch 0, Loss: 1.7601
Epoch 20, Loss: 1.8847
Epoch 40, Loss: 1.6417
Epoch 60, Loss: 1.7583
Epoch 80, Loss: 1.6913
Epoch 100, Loss: 1.6136
Epoch 120, Loss: 1.5813
Epoch 140, Loss: 1.6242
Accuracy: 0.2667


In [76]:
# Check the scores so far

In [77]:
scores

{'Cora': {'GCN': 0.801},
 'Imdb': {'GCN': 0.71},
 'Enzymes': {'GCN': 0.26666666666666666}}

# GIN Performance

#### Node classification

In [78]:
# Helper function to mind max degree in dataset
def find_max_degree(dataset):
    max_deg = -1
    for data in dataset:
        max_deg = max(max_deg, data.edge_index[0].bincount().max().item())
    return max_deg

In [79]:
# Get max of this dataset
cora_max = find_max_degree(cora)

In [80]:
# make a transformed dataset with the one hot encoded degree features
transformed_cora = Planetoid(root='/tmp/Cora', name='Cora', transform=OneHotDegree(cora_max))

In [81]:
#Define a GIN for node classification
class GIN(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.mlp1 = torch.nn.Sequential(
            torch.nn.Linear(transformed_cora.num_node_features, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 16)
        )
        self.mlp2 = torch.nn.Sequential(
            torch.nn.Linear(16, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, transformed_cora.num_classes)
        )
        
        self.conv1 = GINConv(self.mlp1, eps=0.0, train_eps=True)
        self.conv2 = GINConv(self.mlp2, eps=0.0, train_eps=True)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [82]:
# Get the model, data, and optimizer setup in torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GIN().to(device)
data = transformed_cora[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [83]:
# train model for 150 epochs

model.train()
for epoch in range(151):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')

Epoch 000, Loss: 1.9610
Epoch 020, Loss: 0.0504
Epoch 040, Loss: 0.0004
Epoch 060, Loss: 0.0002
Epoch 080, Loss: 0.0003
Epoch 100, Loss: 0.0003
Epoch 120, Loss: 0.0004
Epoch 140, Loss: 0.0005


In [84]:
#evaluate the model
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
scores["Cora"]["GIN"] = acc
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7620


#### Graph Classification

In [85]:
#Define a GIN for node classification
class GIN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super().__init__()

        self.mlp1 = torch.nn.Sequential(
            torch.nn.Linear(num_node_features, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 16)
        )
        self.mlp2 = torch.nn.Sequential(
            torch.nn.Linear(16, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 16)
        )

        self.conv1 = GINConv(self.mlp1, eps=0.0, train_eps=True)
        self.conv2 = GINConv(self.mlp2, eps=0.0, train_eps=True)

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(16, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, num_classes)
        )

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = global_add_pool(x, batch)
        x = self.fc(x)

        return F.log_softmax(x, dim=1)

In [86]:
# get max degrees for our datasets
imdb_max = find_max_degree(imdb)
enzymes_max = find_max_degree(enzymes)

In [87]:
# make a transformed datasets with the one hot encoded degree features
transformed_imdb = TUDataset(root='/tmp/IMDB', name='IMDB-BINARY', transform=OneHotDegree(imdb_max))
transformed_enzymes = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', transform=OneHotDegree(enzymes_max))

In [88]:
# Helper function to load data, train the model, and evaluate it for graph classification
def train_and_evaluate(dataset, epochs = 150):

    # Loading Data
    train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GIN(dataset.num_node_features or 1, dataset.num_classes).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Training
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()

            out = model(data.x, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        if epoch % 20 == 0:
            print(f'Epoch {epoch}, Loss: {loss:.4f}')

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())
        total += data.num_graphs

    accuracy = correct / total
    print(f'Accuracy: {accuracy:.4f}')
    
    return accuracy

In [89]:
train_and_evaluate(transformed_imdb)

Epoch 0, Loss: 0.5298
Epoch 20, Loss: 0.4502
Epoch 40, Loss: 0.3660
Epoch 60, Loss: 0.3644
Epoch 80, Loss: 0.2267
Epoch 100, Loss: 0.3584
Epoch 120, Loss: 0.3621
Epoch 140, Loss: 0.4540
Accuracy: 0.6800


0.68

In [90]:
# Test model on Imdb dataset
scores["Imdb"]["GIN"] = train_and_evaluate(transformed_imdb)

Epoch 0, Loss: 0.5993
Epoch 20, Loss: 0.7544
Epoch 40, Loss: 0.4139
Epoch 60, Loss: 0.3040
Epoch 80, Loss: 0.2289
Epoch 100, Loss: 0.3539
Epoch 120, Loss: 0.1734
Epoch 140, Loss: 0.2893
Accuracy: 0.7100


In [91]:
# Test model on Enzymes dataset
scores["Enzymes"]["GIN"] = train_and_evaluate(transformed_enzymes)

Epoch 0, Loss: 1.9343
Epoch 20, Loss: 1.4890
Epoch 40, Loss: 1.2539
Epoch 60, Loss: 1.2368
Epoch 80, Loss: 0.9421
Epoch 100, Loss: 0.9144
Epoch 120, Loss: 1.0295
Epoch 140, Loss: 0.8362
Accuracy: 0.4000


In [92]:
scores

{'Cora': {'GCN': 0.801, 'GIN': 0.762},
 'Imdb': {'GCN': 0.71, 'GIN': 0.71},
 'Enzymes': {'GCN': 0.26666666666666666, 'GIN': 0.4}}

# GAT Optimal Number of Layers

#### Node classification GCN baseline

In [93]:
# We define a new GCN that can handle a dynamic number of layers
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F.relu(x)
        x = self.convs[-1](x, edge_index)
        
        return F.log_softmax(x, dim=1)

In [94]:
#we turn everything into neat helper functions since we are going to be training a lot of models here

def train_model(model, data, optimizer, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
    
    return model

def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        pred = model(data).argmax(dim=1)
        train_correct = (pred[data.train_mask] == data.y[data.train_mask]).sum()
        val_correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
        test_correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
        
        train_acc = float(train_correct) / int(data.train_mask.sum())
        val_acc = float(val_correct) / int(data.val_mask.sum())
        test_acc = float(test_correct) / int(data.test_mask.sum())
        
    return train_acc, val_acc, test_acc

In [95]:
# Test different numbers of layers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = cora[0].to(device)

results = {}
for num_layers in range(1, 16):
    print(f"Training model with {num_layers} layers")
    
    model = GCN(
        in_channels=cora.num_node_features,
        hidden_channels=16,
        out_channels=cora.num_classes,
        num_layers=num_layers
    ).to(device)
    
    optimizer = Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    
    model = train_model(model, data, optimizer)
    
    train_acc, val_acc, test_acc = evaluate_model(model, data)
    
    results[num_layers] = {
        'train_acc': train_acc,
        'val_acc': val_acc,
        'test_acc': test_acc
    }

print("\n" + "="*50)
print("Summary of Results")
print("="*50)
print("\nNumber of Layers | Train Acc | Val Acc | Test Acc")
print("-"*50)
for layers in range(1, 16):
    r = results[layers]
    print(f"{layers:14d} | {r['train_acc']:.4f} | {r['val_acc']:.4f} | {r['test_acc']:.4f}")

Training model with 1 layers
Training model with 2 layers
Training model with 3 layers
Training model with 4 layers
Training model with 5 layers
Training model with 6 layers
Training model with 7 layers
Training model with 8 layers
Training model with 9 layers
Training model with 10 layers
Training model with 11 layers
Training model with 12 layers
Training model with 13 layers
Training model with 14 layers
Training model with 15 layers

Summary of Results

Number of Layers | Train Acc | Val Acc | Test Acc
--------------------------------------------------
             1 | 1.0000 | 0.7800 | 0.8020
             2 | 1.0000 | 0.7740 | 0.7990
             3 | 1.0000 | 0.7840 | 0.7870
             4 | 1.0000 | 0.7400 | 0.7420
             5 | 1.0000 | 0.7460 | 0.7350
             6 | 1.0000 | 0.7260 | 0.7400
             7 | 1.0000 | 0.7200 | 0.7100
             8 | 1.0000 | 0.6640 | 0.6840
             9 | 0.9000 | 0.5500 | 0.5780
            10 | 0.9714 | 0.5820 | 0.6030
            11 | 

#### Node classification GAT model

In [96]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, heads=4):
        super().__init__()
        
        self.convs = torch.nn.ModuleList()
        
        self.convs.append(GATConv(
            in_channels, 
            hidden_channels, 
            heads=heads, 
            concat=True, 
        ))
        
        hidden_in_channels = hidden_channels * heads
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(
                hidden_in_channels,
                hidden_channels,
                heads=heads,
                concat=True,
            ))
            
        self.convs.append(GATConv(
            hidden_in_channels,
            out_channels,
            heads=heads,
            concat=False,
        ))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for conv in self.convs[:-1]:
            x = F.elu(conv(x, edge_index)) # Note: uses elu instead of relu
        x = self.convs[-1](x, edge_index)
        
        return F.log_softmax(x, dim=1)


In [97]:
# Test different numbers of layers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = cora[0].to(device)

results = {}
for num_layers in range(1, 16):
    print(f"Training model with {num_layers} layers")
    
    model = GAT(
        in_channels=cora.num_node_features,
        hidden_channels=16,
        out_channels=cora.num_classes,
        num_layers=num_layers
    ).to(device)
    
    optimizer = Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    
    model = train_model(model, data, optimizer)
    
    train_acc, val_acc, test_acc = evaluate_model(model, data)
    
    results[num_layers] = {
        'train_acc': train_acc,
        'val_acc': val_acc,
        'test_acc': test_acc
    }

print("\n" + "="*50)
print("Summary of Results")
print("="*50)
print("\nNumber of Layers | Train Acc | Val Acc | Test Acc")
print("-"*50)
for layers in range(1, 16):
    r = results[layers]
    print(f"{layers:14d} | {r['train_acc']:.4f} | {r['val_acc']:.4f} | {r['test_acc']:.4f}")

Training model with 1 layers
Training model with 2 layers
Training model with 3 layers
Training model with 4 layers
Training model with 5 layers
Training model with 6 layers
Training model with 7 layers
Training model with 8 layers
Training model with 9 layers
Training model with 10 layers
Training model with 11 layers
Training model with 12 layers
Training model with 13 layers
Training model with 14 layers
Training model with 15 layers

Summary of Results

Number of Layers | Train Acc | Val Acc | Test Acc
--------------------------------------------------
             1 | 1.0000 | 0.7180 | 0.7690
             2 | 1.0000 | 0.7200 | 0.7520
             3 | 1.0000 | 0.7740 | 0.8000
             4 | 1.0000 | 0.7700 | 0.7820
             5 | 1.0000 | 0.7380 | 0.7610
             6 | 1.0000 | 0.7640 | 0.7480
             7 | 1.0000 | 0.7540 | 0.7570
             8 | 1.0000 | 0.7540 | 0.7760
             9 | 1.0000 | 0.7540 | 0.7790
            10 | 1.0000 | 0.7540 | 0.7710
            11 | 

# GAT Performance

#### Node Classification

In [98]:
class GAT(torch.nn.Module):
    def __init__(self, heads=4):
        super().__init__()
        
        self.conv1 = GATConv(
            cora.num_node_features, 
            16, 
            heads=heads, 
            concat=True,
        )

        self.conv2 = GATConv(
            16 * heads,
            cora.num_classes,
            heads=heads, 
            concat=False,
        )

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.conv1(x, edge_index))  # Note: uses elu not relu
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [99]:
# Get the model, data, and optimizer setup in torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GAT().to(device)
data = cora[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [100]:
# train model for 150 epochs

model.train()
for epoch in range(151):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')

Epoch 000, Loss: 1.9569
Epoch 020, Loss: 0.0012
Epoch 040, Loss: 0.0031
Epoch 060, Loss: 0.0083
Epoch 080, Loss: 0.0066
Epoch 100, Loss: 0.0060
Epoch 120, Loss: 0.0057
Epoch 140, Loss: 0.0054


In [101]:
#evaluate the model
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
scores["Cora"]["GAT"] = acc
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7620


#### Graph Classification

In [114]:
#Define a basic GCN for graph classification
class GAT(torch.nn.Module):
    def __init__(self, num_node_features, num_classes, heads = 4):
        super().__init__()
        self.conv1 = GATConv(
            num_node_features, 
            16, 
            heads=heads, 
            concat=True,
        )
        self.conv2 = GATConv(
            16 * heads,
            16,
            heads=heads, 
            concat=False,
        )
        self.fc = torch.nn.Linear(16, num_classes)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

In [115]:
# Helper fuction to make node features for the IMDB dataset
# We just represent each node with its degree
def create_degree_features(data):
    num_nodes = data.num_nodes
    degrees = torch.bincount(data.edge_index[0], minlength=num_nodes)
    
    feature_vector = degrees.view(-1, 1).float()
    
    return feature_vector

In [118]:
# Helper function to load data, train the model, and evaluate it for graph classification
def train_and_evaluate(dataset, epochs = 150):

    # Loading Data
    train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GAT(dataset.num_node_features or 1, dataset.num_classes).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Training
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()

            # if no node features we add in the degrees
            if data.x is None:
                data.x = create_degree_features(data).to(device)

            out = model(data.x, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        if epoch % 20 == 0:
            print(f'Epoch {epoch}, Loss: {loss:.4f}')

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    for data in test_loader:
        data = data.to(device)

        # if no node features we add in the degrees
        if data.x is None:
            data.x = create_degree_features(data).to(device)

        out = model(data.x, data.edge_index, data.batch)
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())
        total += data.num_graphs

    accuracy = correct / total
    print(f'Accuracy: {accuracy:.4f}')
    
    return accuracy

In [119]:
#Test model on Imdb dataset
scores["Imdb"]["GAT"] = train_and_evaluate(imdb)

Epoch 0, Loss: 0.6996
Epoch 20, Loss: 0.5440
Epoch 40, Loss: 0.5227
Epoch 60, Loss: 0.5801
Epoch 80, Loss: 0.4989
Epoch 100, Loss: 0.5632
Epoch 120, Loss: 0.6200
Epoch 140, Loss: 0.4845
Accuracy: 0.7250


In [120]:
#Test model on Enzymes dataset
scores["Enzymes"]["GAT"] = train_and_evaluate(enzymes)

Epoch 0, Loss: 1.8117
Epoch 20, Loss: 1.6811
Epoch 40, Loss: 1.7913
Epoch 60, Loss: 1.7828
Epoch 80, Loss: 1.6240
Epoch 100, Loss: 1.7801
Epoch 120, Loss: 1.4785
Epoch 140, Loss: 1.4140
Accuracy: 0.3083


# graphGPS Performance

#### Node Classification

In [ ]:
class GPS(torch.nn.Module):
    def __init__(self, heads=1):
        super().__init__()
        nn1 = Sequential(
                Linear(16, 16),
                ReLU(),
                Linear(16, 16),
        )
        nn2 = Sequential(
                Linear(16, 16),
                ReLU(),
                Linear(16, 16),
        )
        self.conv1 = GPSConv(
            channels=16,
            conv=GINEConv(nn1), 
            heads=heads, 
        )
        self.conv2 = GPSConv(
            channels=cora.num_classes,   
            conv=GINEConv(nn2), 
            heads=heads,
        )

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.elu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [181]:
# Get the model, data, and optimizer setup in torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPS().to(device)
data = cora[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

TypeError: __init__() takes 3 positional arguments but 4 were given

In [173]:
# train model for 150 epochs

model.train()
for epoch in range(151):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [ ]:
#evaluate the model
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
scores["Cora"]["GPS"] = acc
print(f'Accuracy: {acc:.4f}')

In [182]:
import argparse
import os.path as osp
from typing import Any, Dict, Optional

import torch
from torch.nn import (
    BatchNorm1d,
    Embedding,
    Linear,
    ModuleList,
    ReLU,
    Sequential,
)
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch_geometric.transforms as T
from torch_geometric.datasets import ZINC
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINEConv, GPSConv, global_add_pool
from torch_geometric.nn.attention import PerformerAttention


#### Graph Classification

In [197]:
class GPS(torch.nn.Module):
    def __init__(self, channels: int, pe_dim: int, num_layers: int,
                 attn_type: str):
        super().__init__()

        self.node_emb = Embedding(28, channels - pe_dim)
        self.pe_lin = Linear(20, pe_dim)
        self.pe_norm = BatchNorm1d(20)
        self.edge_emb = Embedding(4, channels)

        self.convs = ModuleList()
        for _ in range(num_layers):
            nn = Sequential(
                Linear(channels, channels),
                ReLU(),
                Linear(channels, channels),
            )
            conv = GPSConv(channels, GINEConv(nn), heads=4,
                           attn_type=attn_type)
            self.convs.append(conv)

        self.mlp = Sequential(
            Linear(channels, channels // 2),
            ReLU(),
            Linear(channels // 2, channels // 4),
            ReLU(),
            Linear(channels // 4, 1),
        )

    def forward(self, x, pe, edge_index, edge_attr, batch):
        x_pe = self.pe_norm(pe)
        x = torch.cat((self.node_emb(x.squeeze(-1)), self.pe_lin(x_pe)), 1)
        edge_attr = self.edge_emb(edge_attr)

        for conv in self.convs:
            x = conv(x, edge_index, batch, edge_attr=edge_attr)
        x = global_add_pool(x, batch)
        return self.mlp(x)


In [198]:
# Helper fuction to make node features for the IMDB dataset
# We just represent each node with its degree
def create_degree_features(data):
    num_nodes = data.num_nodes
    degrees = torch.bincount(data.edge_index[0], minlength=num_nodes)
    
    feature_vector = degrees.view(-1, 1).float()
    
    return feature_vector

In [203]:
# Helper function to load data, train the model, and evaluate it for graph classification
def train_and_evaluate(dataset, epochs = 150):

    # Loading Data
    train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GPS(16, 16, 2, "multihead").to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Training
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()

            # if no node features we add in the degrees
            if data.x is None:
                data.x = create_degree_features(data).to(device)

            out = model(data.x, data.pe, data.edge_index, data.edge_attr,
                    data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        if epoch % 20 == 0:
            print(f'Epoch {epoch}, Loss: {loss:.4f}')

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    for data in test_loader:
        data = data.to(device)

        # if no node features we add in the degrees
        if data.x is None:
            data.x = create_degree_features(data).to(device)

        out = model(data.x, data.edge_index, data.batch)
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())
        total += data.num_graphs

    accuracy = correct / total
    print(f'Accuracy: {accuracy:.4f}')
    
    return accuracy

In [204]:
train_and_evaluate(imdb)

AttributeError: 'GlobalStorage' object has no attribute 'pe'